In [1]:
import pandas as pd
import numpy as np
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl

In [2]:
os.chdir('G:\\\\Drives compartilhados\\\\República.org\\\\4. Equipes\\\\Dados e Comunicação\\\\DADOS\\\\415 - Repositório de Dados\\\\Repositório Local')


Chamando as bases

In [3]:
df18 = pd.read_excel('MUNIC//Base_MUNIC_2018.xlsx', sheet_name='Educação', usecols=['Cod Municipio','MEDU491','MEDU49212','MEDU49211','MEDU49213'])
df18

,Cod Municipio,MEDU491,MEDU49211,MEDU49212,MEDU49213
0,1100015,Não,Sim,Não,Não
1,1100023,Não,Não,Sim,Não
2,1100031,Não,Sim,Não,Não
3,1100049,Não,Não,Sim,Não
4,1100056,Não,Sim,Não,Não
...,...,...,...,...,...
5565,5222005,Não,Não,Sim,Não
5566,5222054,Não,Sim,Não,Não
5567,5222203,Não,Sim,Não,Não
5568,5222302,Não,Sim,Não,Não


In [4]:
df21 = pd.read_excel('MUNIC//Base_MUNIC_2021.xlsx', sheet_name='Educação', usecols=['CodMun','Medu491','Medu49212','Medu49211','Medu49213'])
df21

,CodMun,Medu491,Medu49211,Medu49212,Medu49213
0,1100015,Não,Não,Sim,Não
1,1100023,Não,Sim,Sim,Não
2,1100031,Não,Sim,Não,Não
3,1100049,Não,Sim,Não,Não
4,1100056,Não,Não,Não,Não
...,...,...,...,...,...
5565,5222005,Não,Não,Sim,Não
5566,5222054,Não,Sim,Não,Não
5567,5222203,Não,Sim,Não,Não
5568,5222302,Não,Sim,Não,Não


In [5]:
uf = pd.read_excel('MUNIC//Base_MUNIC_2021.xlsx', sheet_name = 'Variáveis externas', usecols=[0,1,2,3])
uf

,CodMun,UF,Cod UF,Mun
0,1100015,RO,11,Alta Floresta DOeste
1,1100023,RO,11,Ariquemes
2,1100031,RO,11,Cabixi
3,1100049,RO,11,Cacoal
4,1100056,RO,11,Cerejeiras
...,...,...,...,...
5565,5222005,GO,52,Vianópolis
5566,5222054,GO,52,Vicentinópolis
5567,5222203,GO,52,Vila Boa
5568,5222302,GO,52,Vila Propício


Renomeando as colunas

In [6]:
df18= df18.rename(columns={'Cod Municipio':'id_municipio',
                        'MEDU491':'nomeacao_concurso_publico',
                        'MEDU49212':'nomeacao_eleicao',
                        'MEDU49211' :'nomeacao_indicacao',
                        'MEDU49213':'nomeacao_processo_seletivo'}) 

In [7]:
df21= df21.rename(columns={'CodMun':'id_municipio',
                            'Medu491':'nomeacao_concurso_publico',
                            'Medu49212':'nomeacao_eleicao',
                            'Medu49211' :'nomeacao_indicacao',
                            'Medu49213':'nomeacao_processo_seletivo'}) 

In [8]:
##adicionando a coluna de 'ano' em cada um dos dfs
df18['ano']=2018
df21['ano']=2021 

In [9]:
df = pd.concat([df18,df21]) #juntando os dataframes

In [10]:
uf.columns

Index(['CodMun', 'UF', 'Cod UF', 'Mun'], dtype='object')

In [11]:
x= uf.pivot_table(columns=('CodMun', 'UF','Cod UF', 'Mun'), aggfunc='size')


In [12]:
uf = pd.DataFrame(x).reset_index()[['CodMun', 'UF','Cod UF', 'Mun']]

In [13]:
df = df.merge(uf, right_on='CodMun',left_on='id_municipio') #adicionando siglas e nome das UFs e dos municipios


In [14]:
df

,id_municipio,nomeacao_concurso_publico,nomeacao_indicacao,nomeacao_eleicao,nomeacao_processo_seletivo,ano,CodMun,UF,Cod UF,Mun
0,1100015,Não,Sim,Não,Não,2018,1100015,RO,11,Alta Floresta DOeste
1,1100015,Não,Não,Sim,Não,2021,1100015,RO,11,Alta Floresta DOeste
2,1100023,Não,Não,Sim,Não,2018,1100023,RO,11,Ariquemes
3,1100023,Não,Sim,Sim,Não,2021,1100023,RO,11,Ariquemes
4,1100031,Não,Sim,Não,Não,2018,1100031,RO,11,Cabixi
...,...,...,...,...,...,...,...,...,...,...
11135,5222203,Não,Sim,Não,Não,2021,5222203,GO,52,Vila Boa
11136,5222302,Não,Sim,Não,Não,2018,5222302,GO,52,Vila Propício
11137,5222302,Não,Sim,Não,Não,2021,5222302,GO,52,Vila Propício
11138,5300108,Não,Não,Sim,Não,2018,5300108,DF,53,Brasília


In [15]:
df = df.drop(['CodMun'], axis=1) #eliminando coluna repetida

In [16]:
df= df.rename(columns={'UF':'sigla_uf',
                'Cod UF':'cod_uf',
                'Mun':'nome_municipio'}) #padronizando

In [17]:
df.columns

Index(['id_municipio', 'nomeacao_concurso_publico', 'nomeacao_indicacao',
       'nomeacao_eleicao', 'nomeacao_processo_seletivo', 'ano', 'sigla_uf',
       'cod_uf', 'nome_municipio'],
      dtype='object')

In [36]:
df['nomeacao_eleicao'].unique()

array(['Não', 'Sim', 'Sem dados', 'Recusa'], dtype=object)

In [25]:
df['nomeacao_concurso_publico']=np.where(df['nomeacao_concurso_publico']=='Não informou','Sem dados',df['nomeacao_concurso_publico'])

df['nomeacao_indicacao']=np.where(df['nomeacao_indicacao']=='Não informou','Sem dados',df['nomeacao_indicacao'])
df['nomeacao_indicacao']=np.where(df['nomeacao_indicacao']=='-','Sem dados',df['nomeacao_indicacao'])

df['nomeacao_eleicao']=np.where(df['nomeacao_eleicao']=='Não informou','Sem dados',df['nomeacao_eleicao'])
df['nomeacao_eleicao']=np.where(df['nomeacao_eleicao']=='-','Sem dados',df['nomeacao_eleicao'])

df['nomeacao_processo_seletivo']=np.where(df['nomeacao_processo_seletivo']=='Não informou','Sem dados',df['nomeacao_processo_seletivo'])
df['nomeacao_processo_seletivo']=np.where(df['nomeacao_processo_seletivo']=='-','Sem dados',df['nomeacao_processo_seletivo'])

In [18]:
df = df[['ano','sigla_uf','id_municipio','nome_municipio','nomeacao_concurso_publico', 'nomeacao_indicacao',
       'nomeacao_eleicao', 'nomeacao_processo_seletivo']] #organizando as colunas


In [42]:
df

,ano,sigla_uf,id_municipio,nome_municipio,nomeacao_concurso_publico,nomeacao_indicacao,nomeacao_eleicao,nomeacao_processo_seletivo
0,2018,RO,1100015,Alta Floresta DOeste,Não,Sim,Não,Não
1,2021,RO,1100015,Alta Floresta DOeste,Não,Não,Sim,Não
2,2018,RO,1100023,Ariquemes,Não,Não,Sim,Não
3,2021,RO,1100023,Ariquemes,Não,Sim,Sim,Não
4,2018,RO,1100031,Cabixi,Não,Sim,Não,Não
...,...,...,...,...,...,...,...,...
11135,2021,GO,5222203,Vila Boa,Não,Sim,Não,Não
11136,2018,GO,5222302,Vila Propício,Não,Sim,Não,Não
11137,2021,GO,5222302,Vila Propício,Não,Sim,Não,Não
11138,2018,DF,5300108,Brasília,Não,Não,Sim,Não


Subindo para o GBQ

In [43]:
client = bigquery.Client()
dataset_ref = client.dataset('cargos_lideranca')

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11140 entries, 0 to 11139
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   ano                         11140 non-null  int64 
 1   sigla_uf                    11140 non-null  object
 2   id_municipio                11140 non-null  int64 
 3   nome_municipio              11140 non-null  object
 4   nomeacao_concurso_publico   11140 non-null  object
 5   nomeacao_indicacao          11140 non-null  object
 6   nomeacao_eleicao            11140 non-null  object
 7   nomeacao_processo_seletivo  11140 non-null  object
dtypes: int64(2), object(6)
memory usage: 783.3+ KB


In [45]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano da apuração daquele dado'),
        bigquery.SchemaField('sigla_uf','STRING',description='Sigla da UF'),
        bigquery.SchemaField('id_municipio','INTEGER',description='Identificador do município pelo IBGE'),
        bigquery.SchemaField('nome_municipio','STRING',description='Nome do município da observação'),       
        bigquery.SchemaField('nomeacao_concurso_publico','STRING',description='Se a nomeação de diretor de escola ocorreu por concurso público '),
        bigquery.SchemaField('nomeacao_indicacao','STRING',description='Se a nomeação de diretor de escola ocorreu por indicação'),
        bigquery.SchemaField('nomeacao_eleicao','STRING',description='Se a nomeação de diretor de escola ocorreu por eleição'),
        bigquery.SchemaField('nomeacao_processo_seletivo','STRING',description='Se a nomeação de diretor de escola ocorreu por processo seletivo')

 ]


In [46]:
table_ref = dataset_ref.table('MUNIC_nomeacao_diretor_escola')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df,table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=3ccd30e7-167e-46bb-9eb0-a2695c36f305>